In [1]:
import numpy as np 
import pandas as pd 
import os

# Define train and test paths

In [6]:
train_path='X:/AI/projects/emotionRegonisation/train'
test_path='X:/AI/projects/emotionRegonisation/test'

# Importing Libraries

In [7]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [8]:
# create objects for Data Generation
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/255)

In [9]:
# Augmenting train and test
train_set=train_datagen.flow_from_directory(train_path,
                                             target_size=(72,72),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')
test_set=test_datagen.flow_from_directory(test_path,
                                             target_size=(72,72),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
train_set.class_indices

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [11]:
# See the shape of any data
train_sample=next(train_set)
print(train_sample[0].shape)

(32, 72, 72, 1)


# Model Creation

In [12]:
model=Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), padding='same', activation='relu', input_shape =(72,72,1)))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.30))

model.add(Conv2D(128, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(2, 2), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.30))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.30))

#model.add(Dense(256, activation='tanh'))
#model.add(Dropout(0.25))

#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.25))

    
model.add(Dense(7, activation='softmax'))

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 72, 72, 32)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        8256      
                                                                 
 batch_normalization (BatchN  (None, 72, 72, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 36, 36, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 36, 36, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 36, 128)       3

# Early Stopping

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
early=EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True,verbose=1,min_delta=0.001)

In [ ]:
history=model.fit(train_set,epochs=20,validation_data=test_set,batch_size=32)

Epoch 1/20
898/898 [==============================] - 797s 887ms/step - loss: 1.7166 - accuracy: 0.3060 - val_loss: 1.5882 - val_accuracy: 0.3686
Epoch 2/20
898/898 [==============================] - 794s 884ms/step - loss: 1.6323 - accuracy: 0.3491 - val_loss: 2.1453 - val_accuracy: 0.3291
Epoch 3/20
898/898 [==============================] - 772s 859ms/step - loss: 1.5938 - accuracy: 0.3684 - val_loss: 1.6094 - val_accuracy: 0.3810
Epoch 4/20
898/898 [==============================] - 818s 911ms/step - loss: 1.5559 - accuracy: 0.3875 - val_loss: 1.6275 - val_accuracy: 0.3672
Epoch 5/20
643/898 [====================>.........] - ETA: 3:53 - loss: 1.5316 - accuracy: 0.3988

# Plotting the accuracy and losses epochwise

In [ ]:
# we get the dictionary 'history'
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))

ax[0].plot(history.history['loss'],label='training')
ax[0].plot(history.history['val_loss'],label='validation')
ax[0].legend()
ax[0].set_title('Loss with epochs')

ax[1].plot(history.history['accuracy'],label='training')
ax[1].plot(history.history['val_accuracy'],label='validation')
ax[1].legend()
ax[1].set_title('Accuracy with epochs')

# Saving the model

In [ ]:
model.save('fer_abhigyan.h5')

# Test Image

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
img_path='../input/fer2013/test/happy/PrivateTest_10736786.jpg'
test_image=image.load_img(img_path,target_size=(72,72),color_mode='grayscale')

In [ ]:
type(test_image)

In [ ]:
test_image=image.img_to_array(test_image)

In [ ]:
print(test_image.shape)
plt.imshow(test_image)

## Input to model : (batch_size,row,col,rgb) ---> (1,48,48,1) for 1 image

In [ ]:
test_image=test_image.reshape(1,72,72,1)

# Prediction

In [ ]:
classes=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']

In [ ]:
result=model.predict(test_image)

In [ ]:
result[0]

In [ ]:
y_pred=np.argmax(result[0])
print('The person is ',classes[y_pred])